# Day 9: Disk Fragmenter

He shows you the disk map (your puzzle input) he's already generated. For example:

```
2333133121414131402
```

The disk map uses a dense format to represent the layout of files and free space on the disk. The digits alternate between indicating the length of a file and the length of free space.

So, a disk map like 12345 would represent a one-block file, two blocks of free space, a three-block file, four blocks of free space, and then a five-block file. A disk map like 90909 would represent three nine-block files in a row (with no free space between them).

Each file on disk also has an ID number based on the order of the files as they appear before they are rearranged, starting with ID 0. So, the disk map 12345 has three files: a one-block file with ID 0, a three-block file with ID 1, and a five-block file with ID 2. Using one character for each block where digits are the file ID and . is free space, the disk map 12345 represents these individual blocks:

```
0..111....22222
```
The first example above, `2333133121414131402`, represents these individual blocks:

```
00...111...2...333.44.5555.6666.777.888899
```
The amphipod would like to move file blocks one at a time from the end of the disk to the leftmost free space block (until there are no gaps remaining between file blocks). For the disk map 12345, the process looks like this:
```
0..111....22222
02.111....2222.
022111....222..
0221112...22...
02211122..2....
022111222......
```
The first example requires a few more steps:
```
00...111...2...333.44.5555.6666.777.888899
009..111...2...333.44.5555.6666.777.88889.
0099.111...2...333.44.5555.6666.777.8888..
00998111...2...333.44.5555.6666.777.888...
009981118..2...333.44.5555.6666.777.88....
0099811188.2...333.44.5555.6666.777.8.....
009981118882...333.44.5555.6666.777.......
0099811188827..333.44.5555.6666.77........
00998111888277.333.44.5555.6666.7.........
009981118882777333.44.5555.6666...........
009981118882777333644.5555.666............
00998111888277733364465555.66.............
0099811188827773336446555566..............
```
The final step of this file-compacting process is to update the filesystem checksum. To calculate the checksum, add up the result of multiplying each of these blocks' position with the file ID number it contains. The leftmost block is in position 0. If a block contains free space, skip it instead.

Continuing the first example, the first few blocks' position multiplied by its file ID number are `0 * 0 = 0, 1 * 0 = 0, 2 * 9 = 18, 3 * 9 = 27, 4 * 8 = 32`, and so on. In this example, the checksum is the sum of these, `1928`.

Compact the amphipod's hard drive using the process he requested. What is the resulting filesystem checksum? (Be careful copy/pasting the input for this puzzle; it is a single, very long line.)

In [ ]:
import sys
sys.path.append('..')
from utils import *

inp = get_data(day=9, year=2024)
samp1 = '12345'
samp2 = '2333133121414131402'
samp1, samp2

('12345', '2333133121414131402')

Restate the essential parts of the problem.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Here's the essential problem:

**Input:** A string of digits representing a disk map where digits alternate between:
- File length (at even positions: 0, 2, 4, ...)
- Free space length (at odd positions: 1, 3, 5, ...)

**Files are numbered:** Starting from ID 0 in order of appearance.

**Task:** 
1. Expand the disk map into individual blocks (using file IDs for files, `.` for free space)
2. Move file blocks one at a time from the end to the leftmost free space until no gaps remain
3. Calculate a checksum: sum of (position × file_ID) for all file blocks

**Example:** `12345` becomes `0..111....22222`, which after compacting becomes `022111222......`, with a checksum calculated from the compacted layout.

What part would you like to tackle first?

Let's tackle parsing first (item 1).

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Great choice! Let's think about how to expand the disk map into individual blocks.

For the input `12345`:
- Position 0 (digit `1`): file ID 0, length 1 → `0`
- Position 1 (digit `2`): free space, length 2 → `..`
- Position 2 (digit `3`): file ID 1, length 3 → `111`
- Position 3 (digit `4`): free space, length 4 → `....`
- Position 4 (digit `5`): file ID 2, length 5 → `22222`

Result: `0..111....22222`

How would you approach this? What data structure would you use to store the expanded disk - a string, a list, or something else?

Plan:
1. Split the input to individual digits and convert them to numbers.
2. Split the parsed sequence to two lists - files (even positions) and free space (odd positions).
3. Convert the free space list - repeat '.' for each item in the list.
4. Convert the files list. Enumerate the sequence and repeat the index as a string.
5. Interleave the two lists and join them together.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

That's a solid plan! It breaks down the problem nicely into manageable steps.

One thing to consider: in step 4, when you repeat the index as a string, what happens if a file ID becomes 10 or larger? Would each block still be represented by a single character, or would you need a different approach?

Also, for step 5, when interleaving - the files and free space lists might not be the same length. How will you handle that?

Would you like to start coding step 1, or discuss these edge cases first?

In [ ]:
L(list(samp1))

(#5) ['1','2','3','4','5']

How can I split the list into 2 lists, one containing items at even positions and the other one with items at odd positions?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

You can use Python's slicing with a step! 

For a list `lst`:
- `lst[::2]` gives you items at even positions (0, 2, 4, ...)
- `lst[1::2]` gives you items at odd positions (1, 3, 5, ...)

Try it with your parsed list and see what you get!

In [ ]:
x = L(list(samp1)).map(int)
x[::2], x[1::2]

((#3) [1,3,5], (#2) [2,4])

In [ ]:
x = L(list(samp1)).map(int)
files, gaps = x[::2], x[1::2]
files, gaps

((#3) [1,3,5], (#2) [2,4])

What do you think so far? And remind me where we are overall.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Looking good! You've successfully:
- ✅ Step 1: Split input to digits and converted to numbers
- ✅ Step 2: Split into files (even positions) and free space (odd positions)

Your results `[1,3,5]` for files and `[2,4]` for free space match the example perfectly.

**Where we are overall:** We're working on parsing the disk map (item 1 from your original task list). You've completed the first two steps of your 5-step plan. 

Next up would be steps 3 and 4:
- Step 3: Convert free space numbers to repeated dots
- Step 4: Convert file numbers to repeated file IDs

Ready to tackle step 3 or 4?

In [ ]:
gaps.map(lambda n: [-1]*n)

(#2) [[-1, -1],[-1, -1, -1, -1]]

In [ ]:
def _expand_gaps(gaps): return gaps.map(lambda n: [-1]*n)
exp_gaps = _expand_gaps(gaps)
exp_gaps

(#2) [[-1, -1],[-1, -1, -1, -1]]

In [ ]:
L(str(i)*n for (i, n) in files.enumerate())

(#3) ['0','111','22222']

In [ ]:
def _expand_files(files): return L([i]*n for (i, n) in files.enumerate())
exp_files = _expand_files(files)
exp_files

(#3) [[0],[1, 1, 1],[2, 2, 2, 2, 2]]

How can I interleave the 2 lists?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

You can use `itertools.zip_longest` to handle lists of different lengths, then flatten the result.

Here's a quick approach:

```python
from itertools import zip_longest
L(zip_longest(exp_files, exp_gaps, fillvalue=''))
```

This pairs up items from both lists, using an empty string when one list runs out. Then you'll need to flatten and join. Give it a try and see what you get!

In [ ]:
from itertools import zip_longest

L(zip_longest(exp_files, exp_gaps))

(#3) [([0], [-1, -1]),([1, 1, 1], [-1, -1, -1, -1]),([2, 2, 2, 2, 2], None)]

In [ ]:
L(zip_longest(exp_files, exp_gaps)).concat().concat()

(#15) [0,-1,-1,1,1,1,-1,-1,-1,-1,2,2,2,2,2]

Combine the above steps into a parse function. Please write the full implementation.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

```python
def parse(inp):
    x = L(list(inp)).map(int)
    files, gaps = x[::2], x[1::2]
    
    exp_files = L([i]*n for (i, n) in files.enumerate())
    exp_gaps = L([-1]*n for n in gaps)
    
    return L(zip_longest(exp_files, exp_gaps)).concat().concat()
```

This combines all your steps into one function. Test it with `parse(samp1)` to verify it gives you the expected result!

In [ ]:
def parse(inp):
    x = L(list(inp)).map(int)
    files, gaps = x[::2], x[1::2]
    
    exp_files = L([i]*n for (i, n) in files.enumerate())
    exp_gaps = L([-1]*n for n in gaps)
    
    return L(zip_longest(exp_files, exp_gaps)).concat().concat()
parse(samp1)

(#15) [0,-1,-1,1,1,1,-1,-1,-1,-1,2,2,2,2,2]

In [ ]:
def show(lst):
    return ''.join(L(lst).map(lambda x: '.' if x == -1 else str(x)))
show([0,-1,-1,1,1,1,-1,-1,-1,-1,2,2,2,2,2])

'0..111....22222'

In [ ]:
from fastcore.test import test_eq

test_eq(show(parse(samp1)), '0..111....22222')
test_eq(show(parse(samp2)), '00...111...2...333.44.5555.6666.777.888899')

What do you think so far? And remind me where we are overall.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Excellent work! Your parsing is complete and tested. You've successfully:
- ✅ Parsed the disk map into individual blocks
- ✅ Represented files as their IDs and free space as -1
- ✅ Created a display function and verified with both sample inputs

**Where we are overall:** You've completed the first major task (parsing). The original problem has three main parts:

1. ✅ Expand the disk map into individual blocks 
2. ⏳ Move file blocks from the end to leftmost free space (compacting)
3. ⏳ Calculate the checksum

Ready to tackle the compacting algorithm next?

Let's implement compacting. What would be a good representation for it? I'm afraid that a string will be too slow.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Good thinking about performance! You're already using a list of integers (with -1 for gaps), which is a solid choice. Lists allow efficient indexing and modification.

For the compacting algorithm, you'll need to:
- Find the leftmost gap (first -1)
- Find the rightmost file block (last non -1)
- Swap them
- Repeat until no gaps remain between file blocks

Since you're working with a list, both finding positions and swapping are reasonably fast operations. You could modify the list in-place or create a copy.

How would you approach finding "the leftmost gap" and "the rightmost file block"? What stopping condition would you use?

In [ ]:
p1 = list(parse(samp1))
p1

[0, -1, -1, 1, 1, 1, -1, -1, -1, -1, 2, 2, 2, 2, 2]

In [ ]:
p1.index(-1)

1

In [ ]:
def last_index(lst):
    for i in range(len(lst) - 1, 0, -1):
        if lst[i] != -1:
            return i
last_index(p1)

14

In [ ]:
L(p1).enumerate()

(#15) [(0, 0),(1, -1),(2, -1),(3, 1),(4, 1),(5, 1),(6, -1),(7, -1),(8, -1),(9, -1),(10, 2),(11, 2),(12, 2),(13, 2),(14, 2)]

In [ ]:
i = p1.index(-1)
j = last_index(p1)
i, j

(1, 14)

In [ ]:
p1[i], p1[j] = p1[j], p1[i]
p1


[0, 2, -1, 1, 1, 1, -1, -1, -1, -1, 2, 2, 2, 2, -1]

In [ ]:
while True:
    i, j = p1.index(-1), last_index(p1)
    if j < i: break
    p1[i], p1[j] = p1[j], p1[i]
p1

[0, 2, 2, 1, 1, 1, 2, 2, 2, -1, -1, -1, -1, -1, -1]

In [ ]:
def compact(s):
    p = s.copy()
    while True:
        i, j = p.index(-1), last_index(p)
        if j < i: break
        p[i], p[j] = p[j], p[i]
    return p

s1 = parse(samp1)
show(s1), show(compact(s1))

('0..111....22222', '022111222......')

In [ ]:
def conv(s):
    return L(list(s)).map(lambda c: -1 if c == '.' else int(c))
conv('022111222......')

(#15) [0,2,2,1,1,1,2,2,2,-1,-1,-1,-1,-1,-1]

In [ ]:
compact(s1)

(#15) [0,2,2,1,1,1,2,2,2,-1,-1,-1,-1,-1,-1]

In [ ]:
test_eq(compact(conv('0..111....22222')), conv('022111222......'))
test_eq(compact(conv('00...111...2...333.44.5555.6666.777.888899')), conv('0099811188827773336446555566..............'))

What do you think so far? And remind me where we are overall.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Fantastic progress! Your compacting algorithm works perfectly. You've:
- ✅ Parsed the disk map into individual blocks
- ✅ Implemented and tested the compacting algorithm
- ✅ Verified with both sample inputs

**Where we are overall:** You've completed two of the three main tasks:

1. ✅ Expand the disk map into individual blocks
2. ✅ Move file blocks from the end to leftmost free space (compacting)
3. ⏳ Calculate the checksum

The final step is to calculate the checksum: sum of (position × file_ID) for all file blocks (skipping -1s).

Ready to implement the checksum calculation?

In [ ]:
s = conv('0099811188827773336446555566..............')
s

(#42) [0,0,9,9,8,1,1,1,8,8,8,2,7,7,7,3,3,3,6,4...]

In [ ]:
L(list(s)).enumerate().filter(lambda t: t[1] != -1)

(#28) [(0, 0),(1, 0),(2, 9),(3, 9),(4, 8),(5, 1),(6, 1),(7, 1),(8, 8),(9, 8),(10, 8),(11, 2),(12, 7),(13, 7),(14, 7),(15, 3),(16, 3),(17, 3),(18, 6),(19, 4)...]

In [ ]:
L(list(s)).enumerate().filter(lambda t: t[1] != -1).map(lambda t: t[0] * t[1])

(#28) [0,0,18,27,32,5,6,7,64,72,80,22,84,91,98,45,48,51,108,76...]

In [ ]:
L(list(s)).enumerate().filter(lambda t: t[1] != -1).map(lambda t: t[0] * t[1]).sum()

1928

In [ ]:
from operator import mul

def checksum(s):
    return L(list(s)).enumerate().filter(lambda t: t[1] != -1).starmap(mul).sum()
checksum(s)

1928

In [ ]:
def solve(s):
    p = parse(s)
    c = compact(p)
    return checksum(c)
solve(samp2)

1928

In [ ]:
# solve(inp)

# --- Part Two ---
Upon completion, two things immediately become clear. First, the disk definitely has a lot more contiguous free space, just like the amphipod hoped. Second, the computer is running much more slowly! Maybe introducing all of that file system fragmentation was a bad idea?

The eager amphipod already has a new plan: rather than move individual blocks, he'd like to try compacting the files on his disk by moving whole files instead.

This time, attempt to move whole files to the leftmost span of free space blocks that could fit the file. Attempt to move each file exactly once in order of decreasing file ID number starting with the file with the highest file ID number. If there is no span of free space to the left of a file that is large enough to fit the file, the file does not move.

The first example from above now proceeds differently:

```
00...111...2...333.44.5555.6666.777.888899
0099.111...2...333.44.5555.6666.777.8888..
0099.1117772...333.44.5555.6666.....8888..
0099.111777244.333....5555.6666.....8888..
00992111777.44.333....5555.6666.....8888..
```
The process of updating the filesystem checksum is the same; now, this example's checksum would be 2858.

Start over, now compacting the amphipod's hard drive using this new method instead. What is the resulting filesystem checksum?

Restate the essential parts of the problem.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Here's the essential change for Part Two:

**Different compacting strategy:**
- Instead of moving individual blocks, move **whole files**
- Process files in **decreasing file ID order** (highest ID first)
- Each file attempts to move **exactly once** to the leftmost contiguous free space that can fit it
- If no suitable free space exists to the **left** of the file, it doesn't move

**Example:** File 9 (size 2) moves left first, then file 8 (size 4), then file 7 (size 3), etc.

**Same checksum calculation** as Part One.

How would you approach tracking contiguous spans of files and free space?

It seems that only the compact function needs to be changed.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Exactly right! The `parse` and `checksum` functions can stay the same. Only `compact` needs to change.

For the new `compact`, you'll need to:
1. Identify all files (their ID, starting position, and size)
2. Process them in decreasing ID order
3. For each file, find the leftmost contiguous free space that fits it (and is to the left of the file's current position)
4. Move the entire file if such a space exists

How would you represent or find contiguous spans of free space? And how would you identify each file's position and size?

- I thought about representing files and free space (gaps) as a class Run with position and length attributes.
- Gaps would be represented just by Run objects, files would be `(id, Run)` tuples.
- I'd have two lists - one for gaps and the other one for files.
- Finding a gap with a sufficient size would mean going in the gaps list from left to right and finding a gap with large-enough size.
  - The size and location of the existing gap would have to be updated (the gap itself can stay, size of 0 is fine).
  - The location of the file would update.
  - A new gap will be added in the location of the original file. Since the files are moved right to left, I think I don't need to bother with the compaction of the gaps.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

That's a well-thought-out approach! A few observations:

**Your Run class idea is solid.** Having position and length makes it easy to work with contiguous spans.

**For the gaps list:** When you find a suitable gap and move a file there, you're right that the gap shrinks (or disappears if the file fills it exactly). You'll need to update that gap's position and length.

**One consideration:** When you say "a new gap will be added in the location of the original file" - you're correct that you don't need to merge it with adjacent gaps for the algorithm to work. But make sure when searching for gaps, you're only looking at gaps that are **to the left** of the file's current position.

Would you like to start by implementing the `Run` class, or sketch out the overall structure of the new `compact` function first?

Thinking about it more, maybe the initial construction would be easier from the original representation. It contains the run lengths and to get the positions I'd just need to sum all the previous digits.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

That's a smart insight! Working directly from the original digit representation would be more efficient. You can iterate through the digits, keeping a running sum for positions:

- Even indices → files: create `(file_id, Run(position, length))`
- Odd indices → gaps: create `Run(position, length)`
- Update position by adding the current digit's length after each step

This avoids creating the expanded list first. Would you like to implement the `Run` class and the construction logic, or do you want to discuss the structure a bit more?

Yes, let's do it.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Great! Let's start building the file and gap lists.

First, think about how you'll track the running position. You could:
- Use `itertools.accumulate` on the lengths
- Keep a running variable as you iterate

Which approach feels more natural to you? And what would be a good starting structure - maybe iterate through the digit pairs and build both lists simultaneously?

Let's split the input to a list of digits and then use itertools.accumulate.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Perfect! So you'll have:
- A list of digit lengths
- A list of accumulated positions (starting from 0)

Once you have those two lists, how will you pair them up to create the `Run` objects? Remember you need to handle files (even indices) and gaps (odd indices) differently.

Give the accumulate part a try first and see what you get!

1. Let's split the string to a list of digits and convert them to ints.
2. Then let's use accumulate to keep track of the running positions.
3. Then, let's split the list into files and gaps
4. Finally, add the ids to files (as before).

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Perfect plan! That's a clear sequence of steps.

For step 2, when you use `accumulate`, the result will give you cumulative sums. Think about what the first position should be - does the first file start at position 0, or do you need to offset the accumulated values?

Go ahead and try implementing steps 1 and 2 first, and let's see what you get!

I think the first file starts at position 0.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

You're right! The first file does start at position 0.

So when you use `accumulate`, you'll get cumulative sums of all the lengths. The tricky part is matching these positions with the right files and gaps.

For example, with `12345`:
- Lengths: `[1, 2, 3, 4, 5]`
- After accumulate: what values do you expect?
- File 0 should start at position 0
- Gap should start at position 1 (after file 0)
- File 1 should start at position 3 (after the gap)

Try running `accumulate` and see what adjustments you might need to make to get the starting positions right!

In [ ]:
from itertools import accumulate

L(accumulate([1,2,3,4,5], initial=0))

(#6) [0,1,3,6,10,15]

In [ ]:
from fastcore.basics import store_attr, BasicRepr

class Run(BasicRepr):
    def __init__(self, start, len): store_attr('start,len', self)
Run(0, 5)

Run(start=0, len=5)

In [ ]:
def parse2(inp):
    lengths = L(list(inp)).map(int)
    positions = L(accumulate(lengths, initial=0))
    runs = L(positions, lengths).zip().starmap(Run)
    fs, gaps = runs[::2], runs[1::2]
    files = fs.enumerate()
    return files, gaps

fs, gs = parse2(samp1)
fs, gs

((#3) [(0, Run(start=0, len=1)),(1, Run(start=3, len=3)),(2, Run(start=10, len=5))],
 (#2) [Run(start=1, len=2),Run(start=6, len=4)])

Remind me where we are overall.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Great progress! You've built a new parsing function that represents files and gaps as `Run` objects with positions and lengths.

**Where we are overall:** You're working on Part Two, which requires a different compacting strategy:

1. ✅ Created `Run` class to represent contiguous spans
2. ✅ Built `parse2` to extract files and gaps with their positions/lengths
3. ⏳ Need to implement the new `compact` function that:
   - Processes files in decreasing ID order
   - Moves whole files to the leftmost suitable gap (to the left of current position)
   - Updates gap and file positions accordingly
4. ⏳ Calculate checksum (can reuse existing function, but may need to convert back to the block representation)

Ready to work on the new compact logic?

Yes, let's work on the compaction logic

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Great! Let's think through the compaction algorithm step by step.

You'll need to:
1. Sort files by decreasing ID
2. For each file, find the leftmost gap that fits (if any exists to the left of the file)
3. Move the file and update both the gap list and file position

For step 2, how would you check if a gap is "to the left" of a file? What values would you compare?

In [ ]:
fs

(#3) [(0, Run(start=0, len=1)),(1, Run(start=3, len=3)),(2, Run(start=10, len=5))]

In [ ]:
f = fs[-1]
_, f_run = f
f_run

Run(start=10, len=5)

In [ ]:
gs

(#2) [Run(start=1, len=2),Run(start=6, len=4)]

In [ ]:
from itertools import takewhile

L(takewhile(lambda g: g.start < f_run.start, gs))

(#2) [Run(start=1, len=2),Run(start=6, len=4)]

In [ ]:
gaps2 = L(takewhile(lambda g: g.start < f_run.start, gs))
gaps2

(#2) [Run(start=1, len=2),Run(start=6, len=4)]

In [ ]:
first(gaps2, lambda g: g.len >= f_run.len)

In [ ]:
def find_gap(f_run, gaps):
    gs = L(takewhile(lambda g: g.start < f_run.start, gaps))
    return first(gs, lambda g: g.len >= f_run.len)
find_gap(Run(2,1), [Run(0,2)])

Run(start=0, len=2)

- For each file, find a gap
  - Not found -> continue with a next file
  - Found - update file start, update gap start and len

In [ ]:
def update(f, g):
    f.start = g.start
    g.start += f.len
    g.len -= f.len
f, g = Run(3, 3), Run(0, 3)
update(f, g)
f, g

(Run(start=0, len=3), Run(start=3, len=0))

In [ ]:
fs, gs = parse2(samp2)

fs.reverse()
for f in fs.map(lambda x: x[1]):
    g = find_gap(f, gs)
    if g != None:
        update(f, g)
fs, gs

((#10) [(9, Run(start=2, len=2)),(8, Run(start=36, len=4)),(7, Run(start=8, len=3)),(6, Run(start=27, len=4)),(5, Run(start=22, len=4)),(4, Run(start=12, len=2)),(3, Run(start=15, len=3)),(2, Run(start=4, len=1)),(1, Run(start=5, len=3)),(0, Run(start=0, len=2))],
 (#9) [Run(start=5, len=0),Run(start=11, len=0),Run(start=14, len=1),Run(start=18, len=1),Run(start=21, len=1),Run(start=26, len=1),Run(start=31, len=1),Run(start=35, len=1),Run(start=40, len=0)])

In [ ]:
def file_checksum(f):
    id, run = f
    return L(range(run.start, run.start + run.len)).map(lambda o: o*id).sum()

file_checksum((9, Run(start=2, len=2)))

45

In [ ]:
def solve2(inp):
    fs, gs = parse2(inp)
    fs.reverse()
    for f in fs.map(lambda x: x[1]):
        g = find_gap(f, gs)
        if g != None:
            update(f, g)
    return fs.map(file_checksum).sum()
solve2(samp2)

2858

In [ ]:
solve2(inp)

6349492251099